In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Conv2D, LSTM, MaxPooling2D, CuDNNLSTM, Flatten, Dropout, Reshape, BatchNormalization, PReLU, ELU, LeakyReLU
from keras import optimizers
from keras import initializers
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [3]:
def get_most_frequent(labels):
    
    (values, counts) = np.unique(labels, return_counts=True)
    index = np.argmax(counts)
    return values[index]
    

def segment_data(X_train, y_train, X_val, y_val, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    val_chunks = slidingWindow(X_val, y_val, winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(get_most_frequent(labels))
        
    val_segments = []
    val_labels = []
    for chunk in val_chunks:
        data = chunk[0]
        labels = chunk[1]
        val_segments.append(data)
        val_labels.append(get_most_frequent(labels))
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(get_most_frequent(labels))
        
    return np.array(train_segments), np.array(train_labels), np.array(val_segments), np.array(val_labels), np.array(test_segments), np.array(test_labels)

In [4]:
def prepare_data(train_data, val_data, test_data):

    train_labels = np.array(train_data['labels'].values)
    val_labels = np.array(val_data['labels'].values)
    test_labels = np.array(test_data['labels'].values)
    
    train_data.drop(['labels'], axis=1, inplace=True)
    val_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    
    data = pd.concat([train_data,val_data,test_data])
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(data)
    train_data = scaler.transform(train_data)
    val_data = scaler.transform(val_data)
    test_data = scaler.transform(test_data)
    
    return train_data, val_data, test_data, train_labels, val_labels, test_labels

In [7]:
adl_1_1 = pd.read_csv("../full_dataset/ADL1Opportunity_taskB2_S1.csv",header=None)
adl_1_2 = pd.read_csv("../full_dataset/ADL2Opportunity_taskB2_S1.csv",header=None)
adl_1_3 = pd.read_csv("../full_dataset/ADL3Opportunity_taskB2_S1.csv",header=None)
adl_1_4 = pd.read_csv("../full_dataset/ADL4Opportunity_taskB2_S1.csv",header=None)
adl_1_5 = pd.read_csv("../full_dataset/ADL5Opportunity_taskB2_S1.csv",header=None)
drill_1 = pd.read_csv("../full_dataset/Drill1Opportunity_taskB2.csv",header=None)
adl_2_1 = pd.read_csv("../full_dataset/ADL1Opportunity_taskB2_S2.csv",header=None)
adl_2_2 = pd.read_csv("../full_dataset/ADL2Opportunity_taskB2_S2.csv",header=None)
drill_2 = pd.read_csv("../full_dataset/Drill2Opportunity_taskB2.csv",header=None)
adl_3_1 = pd.read_csv("../full_dataset/ADL1Opportunity_taskB2_S3.csv",header=None)
adl_3_2 = pd.read_csv("../full_dataset/ADL2Opportunity_taskB2_S3.csv",header=None)
drill_3 = pd.read_csv("../full_dataset/Drill3Opportunity_taskB2.csv",header=None)

# import validation data
adl_2_3 = pd.read_csv("../full_dataset/ADL3Opportunity_taskB2_S2.csv",header=None)
adl_3_3 = pd.read_csv("../full_dataset/ADL3Opportunity_taskB2_S3.csv",header=None)

# import test data
adl_2_4 = pd.read_csv("../full_dataset/ADL4Opportunity_taskB2_S2.csv",header=None)
adl_2_5 = pd.read_csv("../full_dataset/ADL5Opportunity_taskB2_S2.csv",header=None)
adl_3_4 = pd.read_csv("../full_dataset/ADL4Opportunity_taskB2_S3.csv",header=None)
adl_3_5 = pd.read_csv("../full_dataset/ADL5Opportunity_taskB2_S3.csv",header=None)

In [8]:
train_frames = [adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1,adl_2_1,adl_2_2,drill_2,adl_3_1,adl_3_2,drill_3]
val_frames = [adl_2_3,adl_3_3]
test_frames = [adl_2_4,adl_2_5,adl_3_4,adl_3_5]
train_data = pd.concat(train_frames)
val_data = pd.concat(val_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
val_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)

In [7]:
print("shapes: train {0}, val {1}, test {2}".format(train_data.shape, val_data.shape, test_data.shape))

shapes: train (497014, 114), val (60949, 114), test (118750, 114)


In [6]:
from sklearn.decomposition import PCA

In [64]:
data = pd.concat([train_data])
data.drop(['labels'], axis=1, inplace=True)

In [65]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,87.0,975.0,-287.0,11.0,1001.0,163.0,95.0,975.0,152.0,195.0,...,-219.0,319.0,-845.0,-20.0,57.0,42.0,57.0,20.0,42.0,175.0
1,124.0,978.0,-389.0,-7.0,1014.0,199.0,124.0,968.0,123.0,226.0,...,-215.0,325.0,-847.0,-17.0,38.0,31.0,38.0,17.0,31.0,175.0
2,102.0,996.0,-440.0,-49.0,1024.0,193.0,127.0,1001.0,113.0,280.0,...,-219.0,328.0,-852.0,27.0,31.0,15.0,31.0,-27.0,15.0,175.0
3,59.0,861.0,-384.0,-9.0,1023.0,202.0,110.0,1007.0,106.0,360.0,...,-221.0,321.0,-852.0,26.0,22.0,-2.0,22.0,-26.0,-2.0,175.0
4,119.0,946.0,-426.0,-22.0,1026.0,188.0,98.0,1001.0,92.0,548.0,...,-213.0,321.0,-850.0,22.0,45.0,-7.0,45.0,-22.0,-7.0,175.0


In [66]:
data_array=data.values;

In [67]:
data_centered = data_array - np.mean(data_array, axis=0)

In [68]:
print(pca.get_covariance().shape)

(113, 113)


In [90]:
def get_k_greatest_indexes_by_variance(data,k):
    variances=[]
    data.rename(columns ={113: 'labels'}, inplace =True)
    data.drop(['labels'], axis=1, inplace=True)
    data_array=data.values;
    data_centered = data_array - np.mean(data_array, axis=0)
    
    for i in range(data_centered.shape[1]):
        variances.append(np.var(data_centered[:,i]))


    k=10
    variances = np.array(variances)

    # only training data
    indexes_by_variance=variances.argsort()
    k_greatest_indexes=indexes_by_variance[-k:]
    print(k_greatest_indexes)
    
    return k_greatest_indexes


data_S1=pd.concat([adl_1_1,adl_1_2,adl_1_3,adl_1_4,adl_1_5,drill_1])

k_greatest_indexes=get_k_greatest_indexes_by_variance(data_S1,k)
data_S2=pd.concat([adl_2_1,adl_2_2,adl_2_3,adl_2_4,adl_2_5,drill_2])
k_greatest_indexes=get_k_greatest_indexes_by_variance(data_S2,k)
data_S3=pd.concat([adl_3_1,adl_3_2,adl_3_3,adl_3_4,adl_3_5,drill_3])
k_greatest_indexes=get_k_greatest_indexes_by_variance(data_S3,k)

[ 92  91  93 111 108  57 106 110  90  94]
[ 58  92  95  77 111 108 106 110  90  94]
[ 93  91  92  95  90  94 111 108 110 106]


In [85]:

new_data=data[k_greatest_indexes]

ValueError: labels ['labels'] not contained in axis

In [ ]:
# full dataset
indexes_by_variance=variances.argsort()
k_greatest_indexes=indexes_by_variance[-k:]
print(k_greatest_indexes)
new_data=data[k_greatest_indexes]


In [44]:
a = np.array([[1, 2], [3, 4]])
print(a)

[[1 2]
 [3 4]]


In [8]:
scaled_train, scaled_val, scaled_test, train_labels, val_labels, test_labels = prepare_data(train_data, val_data, test_data)

In [9]:
print(scaled_train.shape)
print(train_labels.shape)

(497014, 110)
(497014,)


In [10]:
num_sensors = 110
window_size = 24
step_size = 12
classes = 18

In [11]:
train_segments, train_labels, val_segments, val_labels, test_segments, test_labels = segment_data(scaled_train, train_labels, scaled_val, val_labels,
                                                                                                  scaled_test, test_labels, window_size, step_size)

In [12]:
encoder = OneHotEncoder()
train_labels = encoder.fit_transform(train_labels.reshape(-1,1)).toarray()
val_labels = encoder.transform(val_labels.reshape(-1,1)).toarray()
test_labels = encoder.transform(test_labels.reshape(-1,1)).toarray()

In [13]:
# reshape input for CNN
reshaped_train = train_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_val = val_segments.reshape(-1, window_size, num_sensors, 1)
reshaped_test = test_segments.reshape(-1, window_size, num_sensors, 1)

In [ ]:
model = Sequential()

size_of_kernel = (6,1)
kernel_strides = 1
num_filters = 40
dropout_prob = 0.5
inputshape = (window_size, num_sensors, 1)

In [ ]:
model.add(BatchNormalization(input_shape=inputshape))

In [ ]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal', name='1_conv_layer'))
model.add(ELU())

In [ ]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='2_conv_layer'))
model.add(ELU())

In [ ]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='3_conv_layer'))
model.add(ELU())

In [ ]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 kernel_initializer='glorot_normal',
                 name='4_conv_layer'))
model.add(ELU())
model.add(MaxPooling2D(pool_size=(2,1)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(1024,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), name='dense1_layer'))
model.add(ELU())
model.add(Dropout(dropout_prob, name='4_dropout_layer'))

model.add(Dense(512,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1), name='dense2_layer'))
model.add(ELU())
model.add(Dropout(dropout_prob, name='5_dropout_layer'))

model.add(Dense(classes,kernel_initializer='glorot_normal',
                bias_initializer=initializers.Constant(value=0.1),activation='softmax', name='softmax_layer'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

In [ ]:
batchSize = 200
train_epoches = 50

model.fit(reshaped_train,train_labels,validation_data=(reshaped_val,val_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)

train_epoches = 20
all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))
model.fit(all_train,all_labels,validation_data=(reshaped_test,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_DENSE_ELU.h5')

In [14]:
from keras.models import load_model
model = load_model('taskB2_all_Subjects_CNN_DENSE_ELU.h5')
predictions = model.predict(reshaped_test)

all_train = np.concatenate((reshaped_train, reshaped_val))
all_labels = np.concatenate((train_labels, val_labels))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [15]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

Standard accuracy is 0.8971093592076006
0 0.834344046897 0.951943715717
1 0.00576106731352 0.543859649123
2 0.00939963614312 0.802395209581
3 0.00606428138266 0.636363636364
4 0.00798463715383 0.84076433121
5 0.0229431978977 0.632318501171
6 0.0158682029513 0.712230215827
7 0.0101071356378 0.433333333333
8 0.00778249444108 0.468965517241
9 0.00394178289873 0.407407407407
10 0.00424499696786 0.280701754386
11 0.0040428542551 0.4
12 0.00262785526582 0.413793103448
13 0.0067717808773 0.611570247934
14 0.0058621386699 0.549618320611
15 0.0100060642814 0.593548387097
16 0.0319385486153 0.637137989779
17 0.0103092783505 0.31746031746
The weigths sum is 1.0
The computed f1-score is 0.8917615069571931
The f1-score with sklearn function is 0.8917615069571933
Average f1-score is 0.5685228687605002


In [16]:
from keras.models import Model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

layer_name = 'dense2_layer'
inter_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

network_output = inter_layer_model.predict(all_train)

scaler = StandardScaler()

# PCA with certain number of principal components
pca = PCA(n_components=40)
pca.fit(scaler.fit_transform(network_output))
new_output = pca.transform(scaler.transform(network_output))

In [17]:
scaler.transform(network_output)

array([[-0.85612786,  0.21641523, -0.74062717, ..., -0.520172  ,
        -0.10459145,  0.02969447],
       [-1.40582907,  0.02695608, -0.81048703, ..., -0.7195428 ,
        -0.33026576, -0.9444322 ],
       [-1.77317202, -0.06408273, -0.95950562, ..., -0.97115487,
        -0.67366505, -0.30136845],
       ..., 
       [-0.69330859,  0.32248616, -0.92193496, ..., -0.12548703,
        -0.39217541, -0.91126019],
       [-0.73006326,  0.32667246, -0.9358657 , ..., -0.10535894,
        -0.37082985, -0.88008046],
       [-0.95707411,  0.41378313, -0.99942344, ..., -0.06132043,
        -0.28421468, -0.88805932]], dtype=float32)

In [19]:
from sklearn.neighbors import KNeighborsClassifier

svm_labels = np.argmax(all_labels, axis=1)
neigh = KNeighborsClassifier(n_neighbors=18)
neigh.fit(new_output, svm_labels) 

svm_pred = neigh.predict(pca.transform(scaler.transform(inter_layer_model.predict(reshaped_test))))

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(svm_pred, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(svm_pred, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, svm_pred, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

Test accuracy is: 0.8983222154841318
The f1-score with sklearn function is 0.8914608148374094
Average f1-score is 0.5693918707738432


In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

svm_labels = np.argmax(all_labels, axis=1)

# train a one-vs-one multi-class support vector machine
clf = svm.SVC(decision_function_shape='ovr')
clf.fit(new_output, svm_labels)

# predict test data
svm_pred = clf.predict(pca.transform(scaler.transform(inter_layer_model.predict(reshaped_test))))

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(svm_pred, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

scores = cross_val_score(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(svm_pred, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, svm_pred, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

In [ ]:
from sklearn.model_selection import cross_val_predict

predicted = cross_val_predict(clf, pca.transform(inter_layer_model.predict(reshaped_test)),new_test_labels, cv=5)

# measure accuracy and f1-score
num = 0.0
den = 0.0
new_test_labels = np.argmax(test_labels, axis=1)
for pair in zip(predicted, new_test_labels):
    if pair[0] == pair[1]:
        num += 1.0
    
    den += 1.0

print('Test accuracy is: {}'.format(num / den))
f1_scores = f1_score(predicted, new_test_labels, average=None)
print('The f1-score with sklearn function is {}'.format(f1_score(new_test_labels, predicted, average='weighted')))
print('Average f1-score is {}'.format(np.mean(f1_scores)))

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)